Dans ce notebook, on essaye de mettre en place une méthode d'analogie qui exhibe les stéréotypes de genre présentée dans l'article écrit par Bolukbasi. 

Dans les tâches d'analogie standard, on nous donne trois mots, par exemple "he", "she", "king", et on cherche le 4ᵉ mot pour résoudre la relation "he" vers "king" est comme "she" vers "x". Ici, nous modifions la tâche d'analogie de manière à ce qu'avec deux mots donnés, par exemple "he" et "she", nous voulons générer une paire de mots, "x" et "y", telle que la relation "he" vers "x" soit similaire à la relation "she" vers "y" et constitue une bonne analogie. 

Cette modification nous permet de générer systématiquement des paires de mots que l'embedding considère comme analogues à "he" et "she" (ou à toute autre paire de mots de référence).

L'entrée dans notre générateur d'analogies est une paire de mots de départ (a, b) déterminant une direction de départ vect(a) − vect(b) correspondant à la différence normalisée entre les deux mots de départ. Dans la tâche ci-dessous, nous utilisons (a, b) = (she, he). Nous évaluons ensuite toutes les paires de mots x, y en utilisant la métrique suivante :

$S(a,b)(x, y) = \begin{cases} 
\cos(\mathbf{\tilde{a} - \tilde{b}}, \mathbf{\tilde{x} - \tilde{y}}) & \text{if } \|\mathbf{\tilde{x} - \tilde{y}}\| \leq \delta \\
0 & \text{otherwise}
\end{cases}$


où δ est un seuil de similarité. 

L'intuition de la métrique de notation est que nous souhaitons qu'une paire d'analogies soit proche et parallèle à la direction de départ, tandis que les deux mots ne doivent pas être trop éloignés pour être sémantiquement cohérents. 

Le paramètre δ établit le seuil de similarité sémantique. En pratique, il est choisi égal à 1 dans l'article mais **à étudier**. 

C'est parti !!

In [2]:
from gensim import models
import numpy as np

In [3]:
first_model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

KeyedVectors est une classe dans la bibliothèque Gensim qui représente des vecteurs de mots (ou "word vectors"). Elle est utilisée pour stocker et manipuler des embeddings de mots.Dans Gensim, après avoir chargé un modèle Word2Vec pré-entraîné à partir d'un fichier, le modèle est généralement stocké sous forme d'objet KeyedVectors.

In [4]:
first_model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

**Etape 1 :** traitement des données. \
On traite le word embedding selon la même procédure que celle présentée dans l'article. 

In [6]:
vector_size = first_model.vector_size

In [7]:
first_model.sort_by_descending_frequency()

In [8]:
first_model.unit_normalize_all()

On normalise les word vectors. Cette étape peut être utile pour les raisons suivantes : 

1. **Comparaison facilitée :** En normalisant les vecteurs, vous pouvez comparer plus facilement la similarité entre les mots en utilisant des mesures telles que la similarité cosinus. La similarité cosinus entre deux vecteurs normalisés est simplement le produit scalaire des vecteurs, ce qui simplifie les calculs.

2. **Interprétation plus facile :** Les vecteurs normalisés peuvent rendre les interprétations des relations entre les mots plus directes, car la longueur des vecteurs ne joue pas un rôle majeur dans les comparaisons.

In [9]:
first_model.index_to_key[:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

Ensuite, on fait la sélection de mots suivants. À partir des 50 000 mots les plus fréquents, choisissez uniquement les mots en minuscules (pour éviter une distinction artificielle uniquement dû à la casse) et les phrases de moins de 20 caractères en minuscules. Les mots avec des lettres majuscules, des chiffres ou de la ponctuation doivent être exclus. Après cette étape de filtrage, il doit rester 26,377 mots.

In [10]:
def verifier_chaine(chaine) :
    if chaine.islower() and chaine.isalpha() and len(chaine)<20 :
        return True
    return False

Dans Gensim, l'attribut `index_to_key` est utilisé pour accéder à la liste des mots (ou clés) dans le modèle `KeyedVectors`. Cet attribut est un dictionnaire qui mappe les indices numériques attribués à chaque mot aux mots eux-mêmes.

- **Fonctionnement :** Chaque mot dans le modèle Word2Vec est associé à un index numérique unique. L'attribut `index_to_key` est un dictionnaire où les clés sont ces indices numériques et les valeurs sont les mots correspondants.L'ordre de numérotation correspond à l'ordre dans lequel les mots apparaissent dans le corpus. 

- **Exemple :** Supposons que `model.index_to_key` renvoie quelque chose comme `{0: 'chat', 1: 'chien', 2: 'oiseau', ...}`. Cela signifie que le mot "chat" a l'index 0, le mot "chien" a l'index 1, le mot "oiseau" a l'index 2, et ainsi de suite. 

In [11]:
words_sorted = first_model.index_to_key[:50000]
words_sorted = [word for word in words_sorted if verifier_chaine(word)]
words_sorted[:10], len(words_sorted)

(['in', 'for', 'that', 'is', 'on', 'with', 'said', 'was', 'the', 'at'], 24065)

In [12]:
nb_words = len(words_sorted)
print(nb_words)

24065


In [13]:
model = models.keyedvectors.KeyedVectors(vector_size, count=0)

In [14]:
for word in words_sorted :
    model[word] = first_model[word]

In [15]:
model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824870109558105),
 ('teenager', 0.6586930155754089),
 ('girl', 0.5921714305877686),
 ('robber', 0.5585119128227234),
 ('men', 0.5489763021469116),
 ('guy', 0.5420035719871521),
 ('person', 0.5342026352882385),
 ('gentleman', 0.5337990522384644),
 ('motorcyclist', 0.5336882472038269)]

On a fini la première étape de traitement des données. Question reste en suspens : pourquoi je n'ai pas le même nombre de mots dans mon word embedding filtré que dans l'article ?

Maintenant, on passe à l'étape 2. Codons l'algorithme d'analogies de l'article ! 

In [16]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

Les vecteurs de mot ont été normalisés, donc calculer la cosinus-similarité revient à calculer le produit scalaire entre deux vecteurs. Ce produit scalaire peut être calculé par la fonction linear kernel : \
La fonction linear_kernel prend en entrée deux matrices X et Y, où chaque ligne de chaque matrice représente un échantillon (donc dans notre cas, un mot) et chaque colonne représente une caractéristique (une des coordonnées du vecteur associé). Elle renvoie une matrice où l'élément à la position (i, j) est le noyau linéaire entre le i-ème échantillon de la première matrice (X) et le j-ème échantillon de la deuxième matrice (Y). Donc dans notre cas, elle renvoie la similarité entre le mot i de la matrice X et le mot j de la matrice Y. 

In [17]:
test1 = np.ones((3,4))
print(linear_kernel(test1,test1))
#on fait le linear kernel entre deux matrices qui ont 3 mots chacune
#donc on calcule la similarité entre 9 mots différents
#la matrice contient effectivement 9 valeurs
print(cosine_similarity(test1,test1))
#linear kernel et cosine similarity ne sont pas toujours la même chose
#si la matrice n'est pas normalisée, les valeurs ne sont pas les mêmes

[[4. 4. 4.]
 [4. 4. 4.]
 [4. 4. 4.]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [18]:
#on veut une matrice X qui contient les vecteurs de chaque mot
#elle a donc nb_words lignes et vector_size colonnes
X = model.vectors
X.shape

(24065, 300)

Si on fait linear_kernel(X,X), la matrice va nous donner la cosinus_similarité de toutes les paires de mots possibles. Ce sera une matrice symétrique. 

In [19]:
cos_sim = linear_kernel(X)
#quand on ne précise pas de deuxième matrice, la fonction calcule la
#similarité entre chaque ligne de X et elle-même, donc linear_kernel(X,X)
print(cos_sim.shape)
cos_sim[1,2] == cos_sim[2,1]

(24065, 24065)


True

Maintenant qu'on a compris comment fonctionne linear_kernel, appliquons le dans notre algoroithme d'analogies. 

In [20]:
#on trouve les vecteurs associés à she et he 
vect_she = model.get_vector('she')
vect_he = model.get_vector('he')

In [21]:
test = linear_kernel((vect_she-vect_he).reshape(1,-1), X)  
#on calcule la similarité entre le vecteur she-he et tous les autres vecteurs

In [158]:
#maintenant, il faut calculer la similarité entre le vecteur she-he et 
#toutes les différentes possibles entre deux mots
#on va donc faire une matrice de taille nb_words*(nb_words-1)/2

#on crée une matrice de zéros   
sim_mat = np.zeros((int(nb_words*(nb_words-1)/2), 300)) #les vecteurs sont de dimension 300

for i in range(nb_words) :
    for j in range(i+1,nb_words) :
        vect1 = model.get_vector(words_sorted[i])
        vect2 = model.get_vector(words_sorted[j])
        sim_mat[j-i+ nb_words*(i-1)] = (vect1-vect2).reshape(1,-1)

MemoryError: Unable to allocate 647. GiB for an array with shape (289550080, 300) and data type float64

Cette méthode marche pas parce qu'elle demande trop de mémoire. Essayons une autre manière. 

In [22]:
#on vérifie comment on peut calculer la norme d'un vecteur
vecteur_homme = model.get_vector('man')
norme = np.linalg.norm(vecteur_homme)
print(norme)

1.0


In [23]:
#comment obtenir nb de mots contenus dans le word embedding 
nb_mots = len(model.index_to_key)
print(nb_mots)

24065


In [ ]:
def analogies(chosen_model, vect1, vect2, nb_candidats):
    best_candidats = [0]*nb_candidats
    best_scores = np.zeros(nb_candidats)
    dir = vect1 - vect2
    words = chosen_model.index_to_key
    nb_mots = len(words)
    for i in range(nb_mots): 
        for j in range(i+1, nb_mots):  
            mot_x = words[i]
            mot_y = words[j]
            vect_x = chosen_model[mot_x]
            vect_y = chosen_model[mot_y]
            diff = vect_x - vect_y
            if np.linalg.norm(diff) < 1 : 
                sim = cosine_similarity(dir.reshape(1, -1), diff.reshape(1, -1))
                if sim > np.min(best_scores):
                    index = np.argmin(best_scores)
                    best_scores[index] = sim
                    best_candidats[index] = (mot_x, mot_y)
    return best_candidats, best_scores

In [134]:
#on trouve les vecteurs associés à she et he 
vect_she = model.get_vector('she')
vect_he = model.get_vector('he')

In [135]:
word_to_check = 'nuc'

if word_to_check in model:
    print(f"{word_to_check} est présent dans le modèle Word2Vec.")
else:
    print(f"{word_to_check} n'est pas présent dans le modèle Word2Vec.")

nuc n'est pas présent dans le modèle Word2Vec.


In [136]:
results = list(analogies(model, vect_she, vect_he, 10))
print(results)

KeyboardInterrupt: 

Cette méthode est trop lente (toujours pas de résultats au bout de 35mn). Essayons une autre. 

Comme nos vecteurs sont normalisées, on a pour x,y des vecteurs quelconques du word embedding et a,b des vecteurs fixes (qui ici joueront le rôle de he,she) : 

$$ 
\cos(\mathbf{a}-\mathbf{b}, \mathbf{x}-\mathbf{y}) = (\mathbf{a}-\mathbf{b})^T \cdot (\mathbf{x}-\mathbf{y}) = \mathbf{a}^T \cdot \mathbf{x} - \mathbf{b}^T \cdot \mathbf{x} - \mathbf{a}^T \cdot \mathbf{y} + \mathbf{b}^T \cdot \mathbf{y}
$$


Je veux calculer une matrice de similarité mat_sim telle que, en notant i,j les index associés respectivement aux mots x et y, on a : sim_mat(i,j) = cos(a-b,x-y).

In [24]:
#rappel : X = model.vectors
vect_in = model.get_vector('in')
print(model.get_index('in'))

# Vérifier si les matrices sont identiques
are_identical = np.array_equal(vect_in, X[0,:])

if are_identical:
    print("Les matrices sont identiques.")
else:
    print("Les matrices ne sont pas identiques.")

#donc l'index d'un mot correspond à sa ligne dans la matrice X

0
Les matrices sont identiques.


In [25]:
mat_1 = linear_kernel((vect_she-vect_he).reshape(1,-1), X) 
mat_1.shape

(1, 24065)

In [26]:
diff_visée = (vect_she - vect_he)/np.linalg.norm(vect_she - vect_he)

In [27]:
mat_1 = linear_kernel((diff_visée).reshape(1,-1), X) 
#me donne une ligne et 24065 colonnes
#chaque colonne correspond à la cos_sim entre un mot et le vect diff she-he
#attention, il faut normaliser la diff pour pouvoir utilsier linear_kernel

In [30]:
sim_mat = np.zeros((nb_words,nb_words))

xmat_1 = np.array(mat_1[0,:])

sim_mat = np.tile(xmat_1[np.newaxis,:], (nb_words,1))-np.tile(xmat_1[:,np.newaxis], (1,nb_words))

#for i in range(nb_words) :
#    if i%1000 == 0 :
#        print(i, 'mots traités sur', nb_words)
#    for j in range(i+1,nb_words) :
#        sim_mat[i,j] = mat_1[0,i]-mat_1[0,j]


In [31]:
model[0]

array([ 0.05295623,  0.06545979,  0.06619529,  0.04707221,  0.05222073,
       -0.08200861, -0.06141452, -0.11620951,  0.01562944,  0.09929293,
       -0.08568612, -0.028133  ,  0.05222073,  0.05884026, -0.07759558,
       -0.07355032,  0.03328152,  0.07722784, -0.04578507, -0.02721362,
       -0.0342009 ,  0.0356719 , -0.09083465, -0.04817546,  0.00170085,
        0.02794912, -0.00219502,  0.08862814,  0.04652058,  0.04817546,
        0.06104676, -0.05185298, -0.01608913,  0.04155593, -0.06435653,
        0.05185298, -0.09635092, -0.02500711,  0.07428582,  0.13239057,
        0.08347961, -0.02611036, -0.03548803, -0.00638968,  0.02702974,
        0.07759558,  0.02031828, -0.02160541, -0.00386139,  0.08016985,
        0.04504957,  0.07097606,  0.02537486, -0.02041021, -0.07097606,
        0.00077573, -0.03640741,  0.02592649,  0.06104676, -0.08531837,
       -0.06693079,  0.02702974, -0.10958998, -0.1838758 , -0.0463367 ,
        0.03990105,  0.04284306,  0.13533258,  0.04596895,  0.06

In [32]:
sim_mat.shape

(24065, 24065)

In [33]:
mot_0 = model.index_to_key[0]
mot_1 = model.index_to_key[1]
print(mot_0, mot_1)

in for


In [34]:
vect_0 = model.get_vector(mot_0)
vect_1 = model.get_vector(mot_1)
diff1 = vect_0 - vect_1
diff0 = vect_she - vect_he

In [35]:
cos_sim = cosine_similarity(diff1.reshape(1, -1), diff0.reshape(1, -1))[0,0]
print(cos_sim)

-0.010609986


In [36]:
test_1 = sim_mat[0,1]

test_1 == cos_sim, test_1, cos_sim



(False, 0.011060394, -0.010609986)

In [37]:
dir = vect_she - vect_he
n = 9
p = 200
print(model.index_to_key[n])
print(model.index_to_key[p])
diff = model[n] - model[p]
sim = cosine_similarity(dir.reshape(1, -1), diff.reshape(1, -1))[0,0]
print(sim)
print(sim_mat[n,p])
print(sim_mat[n,p]-sim)

at
months
-0.037064265
0.04987895
0.08694322


On observe une légère différence dans les deux calculs de la cos_similarité mais leur différence semble toujours du même signe. On peut espérer que cela n'influe pas trop sur l'ordre des valeurs de coefficient. 

Vérifions si les résultats trouvés sont les mêmes que ceux de l'article. 

Maintenant qu'on a une matrice qui nous donne les similarités, on va chercher les paires de mots associées aux plus gros coefficients, et qui vérifient que norm(x-y) < 1 (cf. condition dans l'article). 

In [43]:
model['in']

array([ 0.05295623,  0.06545979,  0.06619529,  0.04707221,  0.05222073,
       -0.08200861, -0.06141452, -0.11620951,  0.01562944,  0.09929293,
       -0.08568612, -0.028133  ,  0.05222073,  0.05884026, -0.07759558,
       -0.07355032,  0.03328152,  0.07722784, -0.04578507, -0.02721362,
       -0.0342009 ,  0.0356719 , -0.09083465, -0.04817546,  0.00170085,
        0.02794912, -0.00219502,  0.08862814,  0.04652058,  0.04817546,
        0.06104676, -0.05185298, -0.01608913,  0.04155593, -0.06435653,
        0.05185298, -0.09635092, -0.02500711,  0.07428582,  0.13239057,
        0.08347961, -0.02611036, -0.03548803, -0.00638968,  0.02702974,
        0.07759558,  0.02031828, -0.02160541, -0.00386139,  0.08016985,
        0.04504957,  0.07097606,  0.02537486, -0.02041021, -0.07097606,
        0.00077573, -0.03640741,  0.02592649,  0.06104676, -0.08531837,
       -0.06693079,  0.02702974, -0.10958998, -0.1838758 , -0.0463367 ,
        0.03990105,  0.04284306,  0.13533258,  0.04596895,  0.06

Plus efficace : faire un tri des coefs puis parmi les coefs, regardez ceux qui vérifient la condition sur la norme. Pour le tri des coefs, on peut faire un tri partiel : commencer par trier les 10 000 premiers coefs et voir si parmi eux, on peut pas en avoir 10 qui vérifient la condition de norme. Regarder partial sort dans la doc --> fonction argpartition : je commence à trier les 1000 premiers, je fais mon test et si j'ai déjà les 10, je m'arrête. Algo itératif. Il faut mettre la matrice sim_mat à plat, regarder les arguments donnés par argpartition, utiliser unravel pour retrouver le i,j qui correspond à cet indice. 

In [38]:
best_coefs = -np.ones(10)
indices_best = [0]*10
for i in range(nb_words):
    if i%1000 == 0 :
        print(i, 'mots traités sur', nb_words)
    for j in range(i+1, nb_words): 
        if sim_mat[i,j] > np.min(best_coefs):
            if np.linalg.norm(model[i]-model[j]) < 1 : 
                index = np.argmin(best_coefs)
                best_coefs[index] = sim_mat[i,j] 
                indices_best[index] = (i,j)



0 mots traités sur 24065
1000 mots traités sur 24065


KeyboardInterrupt: 

In [240]:
print(best_coefs)
print(indices_best)

[0.45190045 0.48028737 0.45140088 0.47507542 0.44541574 0.4798581
 0.44366646 0.44685441 0.45323342 0.45971811]
[(706, 1414), (4231, 9559), (1001, 5954), (7245, 12936), (706, 7544), (16295, 22583), (1961, 7544), (18506, 20880), (1001, 7544), (1001, 1414)]


In [246]:
for i,j in indices_best :
    analog_she = model.index_to_key[i]
    analog_he = model.index_to_key[j]
    print(analog_she)
    print(analog_he)
    print('')

mother
brother

lady
gentleman

daughter
uncle

queen
kings

mother
nephew

motherhood
fatherhood

sister
nephew

gal
fella

daughter
nephew

daughter
brother



In [242]:
list_analogies = []
for k in range(len(best_coefs)): 
    mot_1 = model.key_to_index[k][0]
    mot_2 = model.key_to_index[k][0]
    list_analogies.append((mot_1, mot_2))   

KeyError: 0